In [1]:

import csv
import matplotlib as mlp
import torch.nn as nn
import numpy as np
import torch.optim as optim
import os
import random
import torch
import PIL
import pandas as pd
import cv2
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
import pathlib
import torch.nn.functional as F
import glob
import matplotlib.pyplot as plt
import shutil
import matplotlib.ticker as ticker
import heapq
from collections import defaultdict
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
X_train = []
Y_train = []
X_val = []
Y_val = []
X_test = []
Y_test = []

In [3]:
tsv_file = open("/kaggle/input/assignment3/aksharantar_sampled/tel/tel_train.csv")
read_tsv = csv.reader(tsv_file)

pad1 = "\t"
space1 = "\n"
for i in read_tsv:   
    Y_train.append(i[1])
    X_train.append(i[0])

X_train = np.array(X_train)

val_tsv_file = open("/kaggle/input/assignment3/aksharantar_sampled/tel/tel_valid.csv")
val_read_tsv = csv.reader(val_tsv_file)
pad2 = "\t"
space2 = "\n"
for i in val_read_tsv:
    Y_val.append(i[1])
    X_val.append(i[0])
X_val= np.array(X_val)

test_tsv_file = open("/kaggle/input/assignment3/aksharantar_sampled/tel/tel_test.csv")
test_read_tsv = csv.reader(test_tsv_file)
pad3 = "\t"
space3 = "\n"
for i in test_read_tsv:
    Y_test.append(i[1])
    X_test.append(i[0])

X_test = np.array(X_test)
trainsize=len(X_train)
testsize = len(X_test)
validationsize = len(X_val)

print("Number of Training samples:", trainsize)
print("Number of Test Samples:" , testsize)
print("Number of Validation Samples:", validationsize)

Number of Training samples: 51200
Number of Test Samples: 4096
Number of Validation Samples: 4096


In [4]:
Y_train = np.array(Y_train)
for i in range(Y_train.shape[0]):
    Y_train[i] = pad1 + Y_train[i] + space1

Y_val = np.array(Y_val)
for i in range(Y_val.shape[0]):
    Y_val[i] = pad2 + Y_val[i] + space2
    
Y_test = np.array(Y_test)
for i in range(Y_test.shape[0]):
    Y_test[i] = pad3+ Y_test[i] + space3


In [5]:
traintargetsize=len(Y_train)
testtargetsize = len(Y_test)
testtargetsize = len(Y_val)
print("TrainSize of Target ",traintargetsize)
print("TrainSize of Target ",testtargetsize)
print("TrainSize of Target ",testtargetsize)


TrainSize of Target  51200
TrainSize of Target  4096
TrainSize of Target  4096


In [6]:
input_corpus = set()
output_corpus = set()
input_corpus = set(char for word in X_train for char in word if char not in input_corpus)
output_corpus = set(char for word in Y_train for char in word if char not in output_corpus)
input_corpus.add(" ")
output_corpus.add(" ")
input_corpus = sorted(list(input_corpus))
output_corpus = sorted(list(output_corpus))
val_input_corpus = set()
val_output_corpus = set()
max_encoder_seq_length = max([len(txt) for txt in X_train]) + 2
max_decoder_seq_length = max([len(txt) for txt in Y_train])

print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)
val_input_corpus = set(char for word in X_val for char in word if char not in val_input_corpus)
val_output_corpus = set(char for word in Y_val for char in word if char not in val_output_corpus)


Max sequence length for inputs: 30
Max sequence length for outputs: 21


In [7]:
num_encoder_tokens = len(input_corpus)
num_decoder_tokens = len(output_corpus)
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)

Number of unique input tokens: 27
Number of unique output tokens: 65


In [8]:


input_data = np.zeros((max_encoder_seq_length,len(X_train)), dtype="int64")
target_data = np.zeros((max_decoder_seq_length,len(X_train)), dtype="int64")
input_data_val = np.zeros((max_encoder_seq_length,len(X_val)), dtype="int64")
target_data_val = np.zeros((max_decoder_seq_length,len(X_val)), dtype="int64")

In [9]:
input_size_encoder = num_encoder_tokens
input_size_decoder = num_decoder_tokens
output_size = num_decoder_tokens


In [10]:
def create_char_index(corpus):
    char_index = defaultdict(int)
    for i, char in enumerate(corpus):
        if char not in char_index:
            char_index[char] = i
    return dict(char_index)

input_char_index = create_char_index(input_corpus)
output_char_index = create_char_index(output_corpus)

In [11]:
for i in range(len(X_train)):
    x = X_train[i]
    y = Y_train[i]
    count=0
    size = x.size
    for t, char in enumerate(x):
        input_data[t, i] = input_char_index[char]
    count=count+1 
    input_data[t + 1 :,i] = input_char_index[" "]
    count1=1
    for t, char in enumerate(y):
        target_data[t, i] = output_char_index[char]
        count1=count+1     
    target_data[t + 1 :,i] = output_char_index[" "]
    

data_type = torch.int64
data_type1=torch.float64
for i in range(len(X_val)):
    x=X_val[i]
    y=Y_val[i]
    count = 0
    size - x.size
    for t, char in enumerate(x):
        input_data_val[t, i] = input_char_index[char]
        count = count +1
    input_data_val[t + 1 :,i] = input_char_index[" "]
    
    count = 0
    for t, char in enumerate(y):
        target_data_val[t, i] = output_char_index[char]
        count = count +1
            
    target_data_val[t + 1 :,i] = output_char_index[" "]

input_data_val = torch.tensor(input_data_val,dtype=data_type)
target_data_val = torch.tensor(target_data_val,dtype=data_type)

In [12]:

input_data = torch.tensor(input_data,dtype=data_type)
reverse_input_char_index = {i: char for char, i in input_char_index.items()}
reverse_target_char_index = {i: char for char, i in output_char_index.items()}
target_data = torch.tensor(target_data,dtype=data_type)

In [13]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout,opsize):
        super(Encoder, self).__init__()
        self.dropout, self.hidden_size, self.num_layers,self.identity_init,self.init_h0 = nn.Dropout(dropout) , hidden_size , num_layers,torch.eye(hidden_size),nn.Linear(num_layers * 2, num_layers)
        self.embedding,self.rnn = nn.Embedding(input_size, embedding_size) , nn.LSTM(embedding_size, hidden_size, num_layers, bidirectional=True) 
        hid_2 =hidden_size*2
        self.fc_hidden, self.fc_cell = nn.Linear(hid_2, hidden_size)  , nn.Linear(hid_2, hidden_size)
    

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size
        a = self.embedding(x)
        embedding = self.dropout(a)
        # embedding shape: (seq_length, N, embedding_size)
        dim_value = 2
        encoder_states, (hidden, cell) = self.rnn(embedding)
        # outputs shape: (seq_length, N, hidden_size)
        hid1 = hidden[0:1]
        hid2 = hidden[1:2]
        hid3 = torch.cat(( hid1, hid2), dim=dim_value)
        hidden = self.fc_hidden(hid3)
        
        cel1 = cell[0:1]
        cel2 = cell[1:2]
        cel3 = torch.cat(( cel1,  cel2), dim=dim_value)
        cell = self.fc_cell(cel3)

        return encoder_states, hidden, cell

In [14]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout):
        super(Decoder, self).__init__()
        self.dropout,self.hidden_size,self.num_layers,self.embedding = nn.Dropout(dropout),hidden_size,num_layers,nn.Embedding(input_size, embedding_size)
        val,zero = 1,0
        hidsize =  hidden_size * 2
        hidemb = hidsize + embedding_size
        self.rnn = nn.LSTM(hidemb, hidden_size, num_layers)       
        hid3 = hidden_size * 3 
        self.energy ,self.fc, self.softmax,self.relu  = nn.Linear(hid3, val),nn.Linear(hidden_size, output_size),nn.Softmax(dim=zero),nn.Hardshrink()

    def forward(self, x, encoder_states, hidden, cell):
        # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        x = x.unsqueeze(0)
        a= self.embedding(x)
        hid = hidden.unsqueeze(0)
        embedding = self.dropout(a)
        length_seq = encoder_states.shape[0]
        # embedding shape: (1, N, embedding_size)
        val  = 2
        sequence_length = encoder_states.shape[0]
        h_reshaped = hidden.repeat(length_seq , 1, 1)

        concat = torch.cat((h_reshaped, encoder_states), dim=val)

        
        attention = (self.softmax(self.relu(self.energy(concat)))).permute(1,2,0)

        

        encoder_states = self.relu(encoder_states.permute(1,0,2))

        
        context_vector = torch.bmm(attention, encoder_states)
        context_vector = context_vector.permute(1,0,2)
        
        rnn_concat = torch.cat((context_vector,embedding),dim=val)
        rnn_input = rnn_concat
       # rnn_input = self.relu(rnn_input)
        outputs, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))
        # outputs shape: (1, N, hidden_size)

        predictions = self.fc(outputs)
        results = self.softmax(predictions)

        # predictions shape: (1, N, length_target_vocabulary) to send it to
        # loss function we want it to be (N, length_target_vocabulary) so we're
        # just gonna remove the first dim
        predictions = predictions.squeeze(0)

        return predictions, hidden, cell, attention,results

In [15]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        x = target[0]
        variables = 0
        target_vocab_size = num_decoder_tokens
        input_vocab_size = num_encoder_tokens
        outputs = torch.zeros(target.shape[0], source.shape[1], target_vocab_size).to(device)

        encoder_states, hidden, cell = self.encoder(source)
        # Grab the first input to the Decoder which will be <SOS> token

        for t in range(1, target.shape[0]):
            # Use previous hidden, cell as context from encoder at start
            output, hidden, cell, _,results = self.decoder(x, encoder_states, hidden, cell)

            # Store next output prediction
            outputs[t] = output
            res = output[0:]
            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess = output.argmax(1)
            tf = teacher_force_ratio
            ran = random.random()
            if ran <  tf:
                x = target[t]
            else:
                x = best_guess

        return outputs

In [16]:
#For RNN
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout,opsize):
        super(EncoderRNN, self).__init__()
        self.dropout, self.hidden_size,self.num_layers = nn.Dropout(dropout),hidden_size,num_layers
        bidirection = True
        self.embedding, self.rnn = nn.Embedding(input_size, embedding_size),nn.RNN(embedding_size, hidden_size, num_layers, bidirectional=True)
        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, bidirectional=bidirection )
        hid_2 = hidden_size * 2
        self.fc_hidden , self.fc_cell = nn.Linear(hid_2, hidden_size) , nn.Linear(hid_2, hidden_size)

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size
        a = self.embedding(x)
        embedding = self.dropout(a)
        # embedding shape: (seq_length, N, embedding_size)
        dim_value = 2
        encoder_states, hidden = self.rnn(embedding)

        hid1 = hidden[0:1]
        hid2 = hidden[1:2]
        hid3 = torch.cat((hidden[0:1], hidden[1:2]), dim=dim_value)
        hidden = self.fc_hidden(hid3)


        return encoder_states, hidden
class DecoderRNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout):
        super(DecoderRNN, self).__init__()
        self.dropout,self.hidden_size,self.num_layers ,self.embedding = nn.Dropout(dropout),hidden_size,num_layers, nn.Embedding(input_size, embedding_size)

        val,zero = 1,0
        hidsize =  hidden_size * 2
        hidemb = hidsize + embedding_size
        self.rnn = nn.RNN(hidemb , hidden_size, num_layers)
        hid_3 = hidden_size * 3
        self.energy, self.fc,self.softmax,self.relu = nn.Linear(hid_3, val),nn.Linear(hidden_size, output_size),nn.Softmax(dim=zero),nn.Hardshrink()


    def forward(self, x, encoder_states, hidden):
        # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        x = x.unsqueeze(0)
        a= self.embedding(x)
        hid = hidden.unsqueeze(0)
        embedding = self.dropout(a)
        length_seq = encoder_states.shape[0]
        # embedding shape: (1, N, embedding_size)
        val  = 2
        sequence_length = encoder_states.shape[0]
        h_reshaped = hidden.repeat(length_seq , 1, 1)
        # h_reshaped: (seq_length, N, hidden_size*2)
        concat = torch.cat((h_reshaped, encoder_states), dim=val)

        
        attention = (self.softmax(self.relu(self.energy(concat)))).permute(1,2,0)
        encoder_states = self.relu(encoder_states.permute(1,0,2))

        
        context_vector = torch.bmm(attention, encoder_states)
        context_vector = context_vector.permute(1,0,2)
        
        rnn_concat = torch.cat((context_vector,embedding),dim=val)
        rnn_input = rnn_concat
           # rnn_input = self.relu(rnn_input)
        outputs, (hidden) = self.rnn(rnn_input, (hidden))
            # outputs shape: (1, N, hidden_size)

        predictions = self.fc(outputs)
        results = self.softmax(predictions)

        predictions = predictions.squeeze(0)

        return predictions,hidden,attention,results                                       
class Seq2SeqGR(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2SeqGR, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        x = target[0]
        variables = 0
        target_vocab_size = num_decoder_tokens
        input_vocab_size = num_encoder_tokens
        outputs = torch.zeros(target.shape[0], source.shape[1], target_vocab_size).to(device)

        encoder_states, hidden = self.encoder(source)
        # Grab the first input to the Decoder which will be <SOS> token

        for t in range(1, target.shape[0]):
            # Use previous hidden, cell as context from encoder at start
            output, hidden, _, results = self.decoder(x, encoder_states, hidden)

            # Store next output prediction
            outputs[t] = output
            res = output[0:]
            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess = output.argmax(1)
            tf = teacher_force_ratio
            ran = random.random()
            if ran <  tf:
                x = target[t]
            else:
                x = best_guess

        return outputs

In [17]:
#For GRU
class EncoderGRU(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout,opsize):
        super(EncoderGRU, self).__init__()
        self.dropout, self.hidden_size,self.num_layers = nn.Dropout(dropout),hidden_size,num_layers
        bidirection = True
        self.embedding, self.rnn = nn.Embedding(input_size, embedding_size),nn.RNN(embedding_size, hidden_size, num_layers, bidirectional=True)
        self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, bidirectional=bidirection )
        hid_2 = hidden_size * 2
        self.fc_hidden , self.fc_cell = nn.Linear(hid_2, hidden_size) , nn.Linear(hid_2, hidden_size)

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size

        a = self.embedding(x)
        embedding = self.dropout(a)
        # embedding shape: (seq_length, N, embedding_size)
        dim_value = 2
        encoder_states, hidden = self.rnn(embedding)

        hid1 = hidden[0:1]
        hid2 = hidden[1:2]
        hid3 = torch.cat((hidden[0:1], hidden[1:2]), dim=dim_value)
        hidden = self.fc_hidden(hid3)


        return encoder_states, hidden
class DecoderGRU(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout):
        super(DecoderGRU, self).__init__()
        self.dropout,self.hidden_size,self.num_layers ,self.embedding = nn.Dropout(dropout),hidden_size,num_layers, nn.Embedding(input_size, embedding_size)

        val,zero = 1,0
        hidsize =  hidden_size * 2
        hidemb = hidsize + embedding_size
        self.rnn = nn.GRU  (hidemb , hidden_size, num_layers)  
        hid_3 = hidden_size * 3
        self.energy, self.fc,self.softmax,self.relu = nn.Linear(hid_3, val),nn.Linear(hidden_size, output_size),nn.Softmax(dim=zero),nn.Hardshrink()


    def forward(self, x, encoder_states, hidden):
        # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        x = x.unsqueeze(0)
        a= self.embedding(x)
        hid = hidden.unsqueeze(0)
        embedding = self.dropout(a)
        length_seq = encoder_states.shape[0]
        # embedding shape: (1, N, embedding_size)
        val  = 2
        sequence_length = encoder_states.shape[0]
        h_reshaped = hidden.repeat(length_seq , 1, 1)
        # h_reshaped: (seq_length, N, hidden_size*2)
        concat = torch.cat((h_reshaped, encoder_states), dim=val)

        
        attention = (self.softmax(self.relu(self.energy(concat)))).permute(1,2,0)

        

        encoder_states = self.relu(encoder_states.permute(1,0,2))

        
        context_vector = torch.bmm(attention, encoder_states)
        context_vector = context_vector.permute(1,0,2)
        
        rnn_concat = torch.cat((context_vector,embedding),dim=val)
        rnn_input = rnn_concat

        outputs, (hidden) = self.rnn(rnn_input, (hidden))
   

        predictions = self.fc(outputs)
        results = self.softmax(predictions)

        predictions = predictions.squeeze(0)

        return predictions,hidden,attention,results  


In [18]:
def beam_find(model, word, norm,input_char_index, output_char_index, reverse_input_char_index,
                reverse_target_char_index, max_encoder_seq_length, max_decoder_seq_length,
                num_encoder_tokens, num_decoder_tokens, beam_width, device, length_penalty=0.6):

    word_t = ''

    word_val = 0
    attention = []
    data = np.zeros((max_encoder_seq_length, 1), dtype="int64")
    datatype = torch.int64
    word_size1 =len(word)
    for t in range(word_size1):
        char = word[t]
        data[t, word_val] = input_char_index[char]
    word_size =len(word)
    data[ word_size:, word_val] = input_char_index[" "]
    data = torch.tensor(data, dtype=datatype).to(device)

    with torch.no_grad():
        encoder_states,hidden,cell= model.encoder(data)

    # Initialize beam
    
    value = 1
    ovalue=-1
    padding = '\t'
    initial_sequence = torch.tensor(np.array(output_char_index[padding]).reshape(value,)).to(device)
    zero = 0
    res =hidden.unsqueeze(0)
    beam = [(0.0, initial_sequence, hidden.unsqueeze(0))]  # [(score, sequence, hidden)]
    start ='\n'
    for _ in range(max_decoder_seq_length):
        candidates = []
        options=[]
        for score, seq, hidden in beam:
            count = 0
            last_token = seq[ovalue].item()
            if last_token == output_char_index[start]:
                # If the sequence ends with the end token, add it to the candidates
                candidates.append((score, seq, hidden))
                count =count+1
                continue
            tempval = np.array(last_token).reshape(value,)
            x = torch.tensor(tempval).to(device)
            output, hidden,cell,at,res = model.decoder(x,encoder_states ,hidden.squeeze(0),cell)
            attention.append(at.detach().cpu().numpy())
            res = output[0:]
            probabilities = F.softmax(output, dim=1)
            result =probabilities
            # Get the top-k probabilities and tokens
            topk_probs, topk_tokens = torch.topk(probabilities, k=beam_width)
            tops = topk_probs[0]
            topk = topk_tokens[0]
            for prob, token in zip(tops, topk):
                new_s= token.unsqueeze(0)
                new_seq = torch.cat((seq, new_s), dim=0)
                new_hidden = hidden.clone().unsqueeze(0)
                penalty = len(new_seq) - 1
                len_pen = length_penalty
                length_penalty_factor = ((penalty) / 5) **  len_pen   # Adjust penalty factor as needed
                candidates.append((score + torch.log(prob).item() / length_penalty_factor, new_seq, new_hidden))
                options.append((score + torch.log(prob).item() / length_penalty_factor))
        # Select top-k candidates based on the accumulated scores
        beam = heapq.nlargest(beam_width, candidates, key=lambda x: x[0])

    # Select the best sequence from the beam as the output
    best_res = ""
    best_score, best_sequence, _ = max(beam, key=lambda x: x[zero])
    word_t = ''.join([reverse_target_char_index[token.item()] for token in best_sequence[value:ovalue]])
    best_res = word_t

    return word_t,attention


In [19]:


import heapq
#     return word_t
def beam_findGR(model, word, norm,input_char_index, output_char_index, reverse_input_char_index,
                reverse_target_char_index, max_encoder_seq_length, max_decoder_seq_length,
                num_encoder_tokens, num_decoder_tokens, beam_width, device, length_penalty=0.6):

    word_t = ''

    word_val = 0
    attention = []
    data = np.zeros((max_encoder_seq_length, 1), dtype="int64")
    datatype = torch.int64
    word_size1 =len(word)
    for t in range(word_size1):
        char = word[t]
        data[t, word_val] = input_char_index[char]
    word_size =len(word)
    data[ word_size:, word_val] = input_char_index[" "]
    data = torch.tensor(data, dtype=datatype).to(device)

    with torch.no_grad():
        encoder_states,hidden = model.encoder(data)

    # Initialize beam
    
    value = 1
    ovalue=-1
    padding = '\t'
    initial_sequence = torch.tensor(np.array(output_char_index[padding]).reshape(value,)).to(device)
    zero = 0
    res =hidden.unsqueeze(0)
    beam = [(0.0, initial_sequence, hidden.unsqueeze(0))]  # [(score, sequence, hidden)]
    start ='\n'
    for _ in range(max_decoder_seq_length):
        candidates = []
        options=[]
        for score, seq, hidden in beam:
            count = 0
            last_token = seq[ovalue].item()
            if last_token == output_char_index[start]:
                # If the sequence ends with the end token, add it to the candidates
                candidates.append((score, seq, hidden))
                count =count+1
                continue
            tempval = np.array(last_token).reshape(value,)
            x = torch.tensor(tempval).to(device)
            output, hidden,at,res = model.decoder(x,encoder_states ,hidden.squeeze(0))
            attention.append(at.detach().cpu().numpy())
            res = output[0:]
            probabilities = F.softmax(output, dim=1)
            result =probabilities
            # Get the top-k probabilities and tokens
            topk_probs, topk_tokens = torch.topk(probabilities, k=beam_width)
            tops = topk_probs[0]
            topk = topk_tokens[0]
            for prob, token in zip(tops, topk):
                new_s= token.unsqueeze(0)
                new_seq = torch.cat((seq, new_s), dim=0)
                new_hidden = hidden.clone().unsqueeze(0)
                penalty = len(new_seq) - 1
                len_pen = length_penalty
                length_penalty_factor = ((penalty) / 5) **  len_pen   # Adjust penalty factor as needed
                candidates.append((score + torch.log(prob).item() / length_penalty_factor, new_seq, new_hidden))
                options.append((score + torch.log(prob).item() / length_penalty_factor))
        # Select top-k candidates based on the accumulated scores
        beam = heapq.nlargest(beam_width, candidates, key=lambda x: x[0])

    # Select the best sequence from the beam as the output
    best_res = ""
    best_score, best_sequence, _ = max(beam, key=lambda x: x[zero])
    word_t = ''.join([reverse_target_char_index[token.item()] for token in best_sequence[value:ovalue]])
    best_res = word_t

    return word_t,attention


In [20]:
def training(num_encoder_tokens,input_embedding_size, dp, cell_type, hidden_size, num_enc_layers, num_dec_layers,num_epochs,output_size,input_size_decoder,batch_size,beam_width):
    
    criterion = nn.CrossEntropyLoss()
    data_train = torch.split(input_data,batch_size,dim=1)
    target_train = torch.split(target_data,batch_size,dim=1)
    learning_rate=0.001
    if(cell_type=="LSTM"):
        encoder_net = Encoder(input_size_encoder,input_embedding_size, hidden_size, num_enc_layers,dp,output_size).to(device)
        decoder_net = Decoder(input_size_decoder,input_embedding_size,hidden_size,output_size,num_dec_layers,dp).to(device)

        model = Seq2Seq(encoder_net, decoder_net).to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    elif(cell_type=="RNN" or cell_type =="GRU"):
        if(cell_type=="RNN"):
            encoder_net = EncoderRNN(input_size_encoder,input_embedding_size, hidden_size, num_enc_layers, dp,output_size).to(device)
            decoder_net = DecoderRNN(input_size_decoder,input_embedding_size,hidden_size,output_size,num_dec_layers,dp).to(device)
        else:
            encoder_net = EncoderGRU(input_size_encoder,input_embedding_size, hidden_size, num_enc_layers, dp,output_size).to(device)
            decoder_net = DecoderGRU(input_size_decoder,input_embedding_size,hidden_size,output_size,num_dec_layers,dp).to(device)
            
        model = Seq2SeqGR(encoder_net, decoder_net).to(device)    
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        #print(train_ds_x)
    total_words_val = len(X_val)
    total_words_test = len(X_test)
        
    for epoch in range(num_epochs):
        correct_prediction = 0
        print(f"[Epoch {epoch} / {num_epochs}]")

        model.eval()
        model.train()
        dataset_Size = len(data_train)
        for i in range(dataset_Size):
            neg = -1
            pos = 1
            x = data_train[i]
            y = target_train[i]
            # Get input and targets and get to cuda
            inp_data = x.to(device)
            target = y.to(device)

            # Forward prop
            output = model(inp_data, target)
            size = output.shape[2]
            temp = output[pos:]
            temp1 =temp.reshape(neg, size)
            optimizer.zero_grad()
            output = temp1
            tar = target[pos:]
            target = tar.reshape(neg)

            
            loss = criterion(output, target)

                # Back prop
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

            # Gradient descent step
            optimizer.step()

        model.eval()
        if(cell_type=="LSTM"):
            for i in range(total_words_val):
       
                decoded_sentence,_ = beam_find(model,X_val[i], 10,input_char_index, output_char_index, reverse_input_char_index, 
                reverse_target_char_index, max_encoder_seq_length, max_decoder_seq_length, 
                num_encoder_tokens, num_decoder_tokens,beam_width,device)
                true_output = Y_val[i][1:-1]
                if true_output== decoded_sentence:
                        correct_prediction = correct_prediction + 1
       
            
        elif(cell_type=="GRU" or cell_type=="RNN"):
            for i in range(total_words_val):
                 decoded_sentence,_ = beam_findGR(model,X_val[i], 10,input_char_index, output_char_index, reverse_input_char_index, 
                 reverse_target_char_index, max_encoder_seq_length, max_decoder_seq_length, 
                 num_encoder_tokens, num_decoder_tokens,beam_width,device)
                 true_output = Y_val[i][1:-1]
                 if true_output== decoded_sentence:
                        correct_prediction = correct_prediction + 1
        test_accuracy = correct_prediction / total_words_val
        print(test_accuracy)
        wandb.log({'val_accuracy_with_attention' : test_accuracy*100})

   

In [21]:
import wandb
wandb.login(key='17d991db26320e751b170877037d1067a164fe6d')
wandb.init(project="Assignment_3_Fina",name="Question_5")

wandb: Currently logged in as: cs22m046 (harshvrma). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [22]:
#training(input_size_encoder,256,0.4,"LSTM",256,1,1,5,num_decoder_tokens,num_decoder_tokens,32,1)

In [23]:
sweep_config2 = {
    'method': 'bayes',
    'metric': {'goal': 'maximize', 'name': 'val_accuracy_with_attention'},
    'parameters': {'embedding_size_input': {'values': [128, 256, 512]},
                   'hidden_size_EncDec': {'values': [128, 256, 512]},
                   'cell_type_value': {'values': ['LSTM','GRU','RNN']},
                   'batch_size_value': {'values': [128,256,512]},
                   'dropout_EncDec': {'values': [0.1, 0.2, 0.3, 0.4]},
                   'Number_Epochs' :{'values':[10,20,30,40]},
                   'beam_width':{'values':[1,2,3,4,5]}
                }}


In [24]:
def train():
    var1 = wandb.init()
    var2 = var1.config
    embedding_size_input = var2.embedding_size_input
    dropout_enc_dec = var2.dropout_EncDec
    cell_type_vl = var2.cell_type_value
    hidden_size_val = var2.hidden_size_EncDec
    number_of_layers = 1
    number_of_epochs = var2.Number_Epochs
    batch_size = var2.batch_size_value
    beam_width = var2.beam_width
    
    training(input_size_encoder , embedding_size_input,  dropout_enc_dec, cell_type_vl, hidden_size_val,  number_of_layers,   number_of_layers,number_of_epochs,num_decoder_tokens,num_decoder_tokens, batch_size,beam_width)

In [ ]:
sweep_id = wandb.sweep(sweep_config2, project="Assignment_3_")
wandb.agent(sweep_id, train, count=5)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: tl7nc3se
Sweep URL: https://wandb.ai/harshvrma/Assignment_3_/sweeps/tl7nc3se


wandb: Waiting for W&B process to finish... (success).
wandb: 🚀 View run Question_5 at: https://wandb.ai/harshvrma/Assignment_3_Fina/runs/qkaml33m
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230521_091204-qkaml33m/logs
wandb: Agent Starting Run: ut5utksv with config:
wandb: 	Number_Epochs: 30
wandb: 	batch_size_value: 512
wandb: 	beam_width: 4
wandb: 	cell_type_value: RNN
wandb: 	dropout_EncDec: 0.1
wandb: 	embedding_size_input: 128
wandb: 	hidden_size_EncDec: 256


[Epoch 0 / 30]
0.008544921875
[Epoch 1 / 30]
0.0380859375
[Epoch 2 / 30]
0.087158203125
[Epoch 3 / 30]
0.123291015625
[Epoch 4 / 30]
0.155517578125
[Epoch 5 / 30]
0.179443359375
[Epoch 6 / 30]
0.2158203125
[Epoch 7 / 30]
0.244140625
[Epoch 8 / 30]
0.2607421875
[Epoch 9 / 30]
0.2587890625
[Epoch 10 / 30]
0.31298828125
[Epoch 11 / 30]
0.31201171875
[Epoch 12 / 30]
0.332275390625
[Epoch 13 / 30]
0.35302734375
[Epoch 14 / 30]
0.3662109375
[Epoch 15 / 30]
0.380126953125
[Epoch 16 / 30]
0.393310546875
[Epoch 17 / 30]
0.3837890625
[Epoch 18 / 30]
0.39599609375
[Epoch 19 / 30]
0.41552734375
[Epoch 20 / 30]
0.406005859375
[Epoch 21 / 30]
0.407470703125
[Epoch 22 / 30]
0.415771484375
[Epoch 23 / 30]
0.41650390625
[Epoch 24 / 30]
0.4248046875
[Epoch 25 / 30]
0.433349609375
[Epoch 26 / 30]
0.4365234375
[Epoch 27 / 30]
0.4365234375
[Epoch 28 / 30]
0.427978515625
[Epoch 29 / 30]
0.432861328125


val_accuracy_with_attention,▁▁▂▃▃▄▄▅▅▅▆▆▆▇▇▇▇▇▇███████████
val_accuracy_with_attention,43.28613


wandb: Agent Starting Run: jb49d6kz with config:
wandb: 	Number_Epochs: 30
wandb: 	batch_size_value: 512
wandb: 	beam_width: 1
wandb: 	cell_type_value: RNN
wandb: 	dropout_EncDec: 0.2
wandb: 	embedding_size_input: 128
wandb: 	hidden_size_EncDec: 128


[Epoch 0 / 30]
0.000244140625
[Epoch 1 / 30]
0.005126953125
[Epoch 2 / 30]
0.01025390625
[Epoch 3 / 30]
0.017333984375
[Epoch 4 / 30]
0.026123046875
[Epoch 5 / 30]
0.0400390625
[Epoch 6 / 30]
0.04296875
[Epoch 7 / 30]
0.0615234375
[Epoch 8 / 30]
0.0673828125
[Epoch 9 / 30]
0.083251953125
[Epoch 10 / 30]
0.09228515625
[Epoch 11 / 30]
0.093994140625
[Epoch 12 / 30]
0.111572265625
[Epoch 13 / 30]
0.11474609375
[Epoch 14 / 30]
0.12744140625
[Epoch 15 / 30]
0.136962890625
[Epoch 16 / 30]
0.136962890625
[Epoch 17 / 30]
0.154052734375
[Epoch 18 / 30]
0.126708984375
[Epoch 19 / 30]
0.169189453125
[Epoch 20 / 30]
0.1279296875
[Epoch 21 / 30]
0.181396484375
[Epoch 22 / 30]
0.1728515625
[Epoch 23 / 30]
0.1884765625
[Epoch 24 / 30]
0.2119140625
[Epoch 25 / 30]
0.192626953125
[Epoch 26 / 30]
0.215087890625
[Epoch 27 / 30]
0.236328125
[Epoch 28 / 30]
0.21142578125
[Epoch 29 / 30]
0.2392578125


val_accuracy_with_attention,▁▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▅▆▅▆▆▇▇▇▇█▇█
val_accuracy_with_attention,23.92578


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 6tnk8tgg with config:
wandb: 	Number_Epochs: 20
wandb: 	batch_size_value: 512
wandb: 	beam_width: 5
wandb: 	cell_type_value: RNN
wandb: 	dropout_EncDec: 0.4
wandb: 	embedding_size_input: 256
wandb: 	hidden_size_EncDec: 128


[Epoch 0 / 20]
0.0
[Epoch 1 / 20]
0.001220703125
[Epoch 2 / 20]
0.008544921875
[Epoch 3 / 20]
0.0205078125
[Epoch 4 / 20]
0.029296875
[Epoch 5 / 20]
0.043212890625
[Epoch 6 / 20]
0.061279296875
[Epoch 7 / 20]
0.072998046875
[Epoch 8 / 20]
0.083984375
[Epoch 9 / 20]
0.109375
[Epoch 10 / 20]
0.11865234375
[Epoch 11 / 20]
0.123291015625
[Epoch 12 / 20]
0.140625
[Epoch 13 / 20]
0.148193359375
[Epoch 14 / 20]
0.162841796875
[Epoch 15 / 20]
0.172119140625
[Epoch 16 / 20]
0.181396484375
[Epoch 17 / 20]
0.18701171875
[Epoch 18 / 20]
